In [1]:
import brightway2 as bw
import numpy as np
from pathlib import Path
import pickle, json
from pypardiso import spsolve

#Local files
from import_databases import *
from utils_exiobase import exiobaseLCA

# 1. Get exiobase scores

In [2]:
bw.projects.set_current("exiobase_industry_workaround")
exiobase_name = "EXIOBASE 2.2"
ex = bw.Database(exiobase_name)
methods = [
    ('IPCC 2013', 'climate change', 'GTP 100a'),
]

In [3]:
%%time
# takes 20 min or so
filepath = Path("exiobase_lca.pickle")
if not filepath.exists():
    exiobase_scores_industry_workaround = {}
    for i,act in enumerate(ex):
        if i%500==0:
            print("lca for act {0:4d}/{1}".format(i,len(ex)))
        db_code = (act.as_dict()["database"], act.as_dict()["code"])
        demand = {act: 1}
        lca = bw.LCA(demand, methods[0])
        lca.lci()
        lca.lcia()
        exiobase_scores_industry_workaround[db_code] = {}
        for method in methods:
            lca.switch_method(method)
            lca.redo_lcia()
            exiobase_scores_industry_workaround[db_code].update({
                method: lca.score
            })
    with open(filepath, 'wb') as f:
        pickle.dump(exiobase_scores_industry_workaround, f)
else:
    with open(filepath, 'rb') as f:
        exiobase_scores_industry_workaround = pickle.load(f)

CPU times: user 5.65 ms, sys: 2.02 ms, total: 7.67 ms
Wall time: 7.74 ms


# 2. Import databases

In [4]:
which_pc = "sasha"

project = "rebound"
bw.projects.set_current(project)
del bw.databases['CH consumption 1.0']

In [5]:
if which_pc == 'sasha':
    # Older databases
    ei33_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.3 cutoff/datasets'  
    ex22_path = '/Users/akim/Documents/LCA_files/exiobase 2.2/mrIOT_IxI_fpa_coefficient_version2.2.2'
    ag12_path = '/Users/akim/Documents/LCA_files/agribalyse 1.2/AGRIBALYSEv1.2_Complete.xml'
    # Newer databases
    ei36_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.6 cutoff/datasets'  
    # ex34_path = '/Users/akim/Documents/LCA_files/exiobase 3.4/IOT_2011_pxp/' #TODO
    ag13_path = '/Users/akim/Documents/LCA_files/agribalyse 1.3/Agribalyse CSV FINAL_no links_Nov2016v3.CSV'
    habe_path = '/Users/akim/Documents/LCA_files/HABE_2017/'
    co_path = 'data/es8b01452_si_002.xlsx'

In [6]:
%%time 
# This takes a while
ei36_name = 'ecoinvent 3.6 cutoff'
co_name = CONSUMPTION_DB_NAME

if project == 'rebound':
    bw.bw2setup()
    import_exiobase_22(ex22_path)
    import_ecoinvent(ei36_path, ei36_name)
    import_agribalyse_13(ag13_path, ei36_name)
    # Consumption DB
    create_ecoinvent_33_project(ei33_path)
    exclude_dbs = [
        'heia', 
    ]
    import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs)
    add_consumption_activities(co_name, habe_path)

Biosphere database already present!!! No setup is needed
EXIOBASE 2.2 database already present!!! No import is needed
ecoinvent 3.6 cutoff database already present!!! No import is needed
Agribalyse 1.3 - ecoinvent 3.6 cutoff database already present!!! No import is needed
Biosphere database already present!!! No setup is needed
ecoinvent 3.3 cutoff database already present!!! No import is needed
Extracted 1 worksheets in 0.39 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_act

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 11/27/2020 20:47:56
  Finished: 11/27/2020 20:47:57
  Total time elapsed: 00:00:00
  CPU %: 98.50
  Memory %: 1.88
Created database: CH consumption 1.0
CPU times: user 1min 7s, sys: 7.64 s, total: 1min 15s
Wall time: 1min 16s


# 3. Compute ecoinvent and agribalyse scores

In [8]:
co = bw.Database(CONSUMPTION_DB_NAME)
demand_act = co.search('average consumption')
assert len(demand_act) == 1
demand = {demand_act[0]: 1}

In [9]:
%%time
# This takes 30s or so
exio_lca = exiobaseLCA(
    project, 
    demand,
    exiobase_scores_industry_workaround,
)
scores = exio_lca.compute_total_scores()
scores

CPU times: user 38.7 s, sys: 5.62 s, total: 44.3 s
Wall time: 26 s


{('IPCC 2013', 'climate change', 'GTP 100a'): 1709.0750836479817}

# Contribution analysis

In [10]:
%%time
project = 'rebound'
co = bw.Database("CH consumption 1.0")
contribution_scores = {}
i = 0
for act in list(co):
    demand = {act: 1}
    exio_lca = exiobaseLCA(
        project, 
        demand,
        exiobase_scores_industry_workaround,
    )
    act_name = act["name"]
    score = exio_lca.compute_total_scores()
    contribution_scores[act] = score
    print(i, score[methods[0]], act)
    i+=1

0 0.4780347510909685 'Beans and peas' (kilogram, CH, None)
1 0.29370403056054595 'Ready-made foods' (kilogram, CH, None)
2 0.08717017210589198 'Potatoes' (kilogram, CH, None)
3 0.168382280573137 'Oranges and other citrus fruits' (kilogram, CH, None)
4 0.23727655407762813 'Photographic and cinematographic equipment and optical instruments' (CHF, CH, None)
5 0.4819895914867959 'Berries' (kilogram, CH, None)
6 0.09120903419317959 'Regular maintenance costs of principal residence' (CHF, CH, None)
7 0.460480293489614 'Tools and equipment without engines for house and garden' (CHF, CH, None)
8 0.5438711745985397 'Medical products, therapeutic devices and products' (CHF, CH, None)
9 0.460480293489614 'Other household utensils' (CHF, CH, None)
10 0.16139358867213643 'Sports and crafting classes' (CHF, CH, None)
11 0.25633247871543474 'Equipment for the reception, recording and reproduction of sound and pictures' (CHF, CH, None)
12 0.46690707041943885 'Glassware, tableware' (CHF, CH, None)
13 3

115 0.16139358867213643 'Entries in sports facilities' (CHF, CH, None)
116 0.09120903419317959 'Domestic services and household services' (CHF, CH, None)
117 0.10533112796493806 'Soups and bouillons' (kilogram, CH, None)
118 0.07076793235305774 'Natural gas and other fuels of principal residence' (megajoule, CH, None)
119 0.16139358867213643 'Fees for radio and TV reception' (CHF, CH, None)
120 0.11062665668240976 'Cleaning, repair and hire of clothing' (CHF, CH, None)
121 0.16139358867213643 'Dues for sports clubs and associations' (CHF, CH, None)
122 0.3215805622451814 'Non-durable products of cotton and paper for personal care' (CHF, CH, None)
123 0.30855488452753754 'Sewing and knitting products' (CHF, CH, None)
124 0.23535154313569895 'Books and brochures' (CHF, CH, None)
125 1.4237950274448177 'Yoghurt' (kilogram, CH, None)
126 10.934107294916274 'Veal' (kilogram, CH, None)
127 9.382629015653418 'Fresh, soft and melted cheese' (kilogram, CH, None)
128 2.414801888021446 'Poultry, 

In [12]:
filepath = "contribution_scores.pickle"
with open(filepath, 'wb') as f:
    pickle.dump(contribution_scores, f)

In [13]:
contribution_scores

{'Beans and peas' (kilogram, CH, None): {('IPCC 2013',
   'climate change',
   'GTP 100a'): 0.4780347510909685},
 'Ready-made foods' (kilogram, CH, None): {('IPCC 2013',
   'climate change',
   'GTP 100a'): 0.29370403056054595},
 'Potatoes' (kilogram, CH, None): {('IPCC 2013',
   'climate change',
   'GTP 100a'): 0.08717017210589198},
 'Oranges and other citrus fruits' (kilogram, CH, None): {('IPCC 2013',
   'climate change',
   'GTP 100a'): 0.168382280573137},
 'Photographic and cinematographic equipment and optical instruments' (CHF, CH, None): {('IPCC 2013',
   'climate change',
   'GTP 100a'): 0.23727655407762813},
 'Berries' (kilogram, CH, None): {('IPCC 2013',
   'climate change',
   'GTP 100a'): 0.4819895914867959},
 'Regular maintenance costs of principal residence' (CHF, CH, None): {('IPCC 2013',
   'climate change',
   'GTP 100a'): 0.09120903419317959},
 'Tools and equipment without engines for house and garden' (CHF, CH, None): {('IPCC 2013',
   'climate change',
   'GTP 100